In [28]:
sc = SparkContext()

In [19]:
import pyspark 
from pyspark import SparkContext, SparkConf
from csv import reader
from glob import glob
from operator import add
from datetime import datetime
import os

In [29]:
complaint = sc.textFile('../data/311.csv')

In [30]:
complaints = complaint.mapPartitions(lambda x: reader(x, delimiter = ',',quotechar = '"'))

In [31]:
header = complaints.take(2)
print header

[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name', 'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip', 'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2', 'Address Type', 'City', 'Landmark', 'Facility Type', 'Status', 'Due Date', 'Resolution Action Updated Date', 'Community Board', 'Borough', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough', 'School Name', 'School Number', 'School Region', 'School Code', 'School Phone Number', 'School Address', 'School City', 'School State', 'School Zip', 'School Not Found', 'School or Citywide Complaint', 'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 'Garage Lot Name', 'Ferry Direction', 'Ferry Terminal Name', 'Latitude', 'Longitude', 'Location'], ['35603714', '03/01/2017 12:00:00 AM', '03/13/201

In [41]:
import numpy as np
from collections import Counter

In [7]:
index = Counter()
for i,j in enumerate(header[0]):
    index[j] = i
print index

Counter({'Location': 51, 'Longitude': 50, 'Latitude': 49, 'Ferry Terminal Name': 48, 'Ferry Direction': 47, 'Garage Lot Name': 46, 'Bridge Highway Segment': 45, 'Road Ramp': 44, 'Bridge Highway Direction': 43, 'Bridge Highway Name': 42, 'Taxi Pick Up Location': 41, 'Taxi Company Borough': 40, 'Vehicle Type': 39, 'School or Citywide Complaint': 38, 'School Not Found': 37, 'School Zip': 36, 'School State': 35, 'School City': 34, 'School Address': 33, 'School Phone Number': 32, 'School Code': 31, 'School Region': 30, 'School Number': 29, 'School Name': 28, 'Park Borough': 27, 'Park Facility Name': 26, 'Y Coordinate (State Plane)': 25, 'X Coordinate (State Plane)': 24, 'Borough': 23, 'Community Board': 22, 'Resolution Action Updated Date': 21, 'Due Date': 20, 'Status': 19, 'Facility Type': 18, 'Landmark': 17, 'City': 16, 'Address Type': 15, 'Intersection Street 2': 14, 'Intersection Street 1': 13, 'Cross Street 2': 12, 'Cross Street 1': 11, 'Street Name': 10, 'Incident Address': 9, 'Incide

In [32]:
complaints = complaint.map(lambda x: x.encode('ascii', 'ignore')) \
                      .mapPartitions(lambda x: reader(x, delimiter = ',',quotechar = '"'))
complaints.take(2)

[['Unique Key',
  'Created Date',
  'Closed Date',
  'Agency',
  'Agency Name',
  'Complaint Type',
  'Descriptor',
  'Location Type',
  'Incident Zip',
  'Incident Address',
  'Street Name',
  'Cross Street 1',
  'Cross Street 2',
  'Intersection Street 1',
  'Intersection Street 2',
  'Address Type',
  'City',
  'Landmark',
  'Facility Type',
  'Status',
  'Due Date',
  'Resolution Action Updated Date',
  'Community Board',
  'Borough',
  'X Coordinate (State Plane)',
  'Y Coordinate (State Plane)',
  'Park Facility Name',
  'Park Borough',
  'School Name',
  'School Number',
  'School Region',
  'School Code',
  'School Phone Number',
  'School Address',
  'School City',
  'School State',
  'School Zip',
  'School Not Found',
  'School or Citywide Complaint',
  'Vehicle Type',
  'Taxi Company Borough',
  'Taxi Pick Up Location',
  'Bridge Highway Name',
  'Bridge Highway Direction',
  'Road Ramp',
  'Bridge Highway Segment',
  'Garage Lot Name',
  'Ferry Direction',
  'Ferry Termina

In [52]:
def check_datetime(dt):
    try:
        d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
        if d.year < 2009 or d.year > 2017:
            return False
    except ValueError:
        return False

    return True

def get_date(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return str(d.year), str(d.month), str(d.day)


def get_date2(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return str(d.date())

def solve_time(created, closed):
    created = datetime.strptime(created, "%m/%d/%Y %I:%M:%S %p")
    try:
        closed = datetime.strptime(closed, "%m/%d/%Y %I:%M:%S %p")
    except ValueError:
        closed = datetime.now()
    return (closed - created).days

def get_week(dt):
    d = datetime.strptime(dt, "%m/%d/%Y %I:%M:%S %p")
    return d.year, d.weekday(), d.hour

In [24]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_date(x[1]), x[3],x[5]), 1)) \
          .foldByKey(0, add) \
          .take(1)

[(((2017, 3, 8), 'HPD', 'GENERAL'), 78)]

In [59]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_date(x[1])[0:2], x[3],x[5],x[8],x[22],x[23]), 1)) \
          .foldByKey(0, add) \
          .count()

1557687

In [37]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x: ((x[3],x[5],x[8],x[22],x[23]), solve_time(x[1], x[2]))) \
          .groupByKey() \
          .mapValues(list).take(2)
          

[(('DSNY', 'Overflowing Litter Baskets', '', '17 BROOKLYN', 'BROOKLYN'), [1]),
 (('DSNY', 'Dirty Conditions', '10013', '01 MANHATTAN', 'MANHATTAN'),
  [4, 3, 5, 3, 3])]

In [45]:
complaints.filter(lambda x : check_datetime(x[1])) \
          .map(lambda x : ((get_week(x[1]),x[3],x[5],x[23]), 1)) \
          .foldByKey(0, add) \
          .take(3)

[(((2017, 6, 19), 'NYPD', 'Noise - Residential', 'BROOKLYN'), 28),
 (((2017, 6, 10), 'NYPD', 'Noise - Vehicle', 'BRONX'), 2),
 (((2017, 4, 11), 'DPR', 'New Tree Request', 'QUEENS'), 2)]

In [54]:
## number of complaint by agency each day in years
num_agency_day = complaints.filter(lambda x : check_datetime(x[1])) \
                           .map(lambda x : ((get_date2(x[1]),x[3]), 1)) \
                           .foldByKey(0, add) \
                           .map(lambda x: [x[0][0],x[0][1],x[1]]).collect()

In [55]:
num_agency_day = np.array(num_agency_day)

In [51]:
d = datetime.strptime('03/31/2017 01:59:21 PM', "%m/%d/%Y %I:%M:%S %p")
str(d.date())

'2017-03-31'

In [10]:
import plotly.plotly as py
from plotly.offline import plot
import plotly.graph_objs as go


In [11]:
plot([go.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph.html',image='jpeg')

'file:///Users/xiaodiu/Documents/github/ds1004/Notes/my-graph.html'

In [16]:
 trace = go.Scatter (
        x = [ 1, 2, 3 ] , 
        y = [ 1, 2, 3 ] , 
        marker = dict (
            color = [ 'red', 'blue' , 'green' ],
            size = [ 30, 80, 200 ] ) ,
        mode = 'markers')
plot([trace], filename='bubble.html')

'file:///Users/xiaodiu/Documents/github/ds1004/Notes/bubble.html'

In [17]:
trace = go.Scatter(
        x=[1,2],
        y=[1,2],
        fill = 'tonexty')
data = [trace]
plot([trace], filename='bubble.html')


'file:///Users/xiaodiu/Documents/github/ds1004/Notes/bubble.html'

In [60]:
np.where(num_agency_day == 'ACS')[0]

array([ 10,  21,  29,  47, 113, 133, 138, 140, 184, 206, 207, 247, 250,
       282, 286, 311, 400, 420, 429, 552, 554, 575, 599])